<a href="https://colab.research.google.com/github/rutkovskii/FlightAnalysis/blob/main/RusAirtraffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re
from datetime import datetime

import matplotlib.pyplot as plt


def df_printer(df):
  with pd.option_context('display.max_rows', None,):
    print(df)

# Exctracting Data

###Working with Flight Data

In [ ]:
# Read all flights for January and February 2022

# Source: https://zenodo.org/record/6325961#.YkHzUzdBxQI

flights_jan = pd.read_csv('/content/drive/MyDrive/FlightAnalysis/flightlist_20220101_20220131.csv')#('/content/flightlist_20220101_20220131.csv.gz')
flights_feb = pd.read_csv('/content/drive/MyDrive/FlightAnalysis/flightlist_20220201_20220228.csv')#('/content/flightlist_20220201_20220228.csv.gz')

In [ ]:
# Sneakpeak
flights_jan.head()

In [ ]:
# Attempt to find Pulkovo Airport in this data set 
# Origins and Destinations are written in ICAO codes
# ICAO — International Civil Aviation Organization

# Pulkovo's ICAO code: ULLI
flights_jan.loc[flights_jan['origin'] == "ULLI"].head(10)


### Extracting International Airports and the Page 

In [ ]:
# Getting tables with all international airports
url = "https://en.wikipedia.org/wiki/List_of_international_airports_by_country"
airport_dfs = pd.read_html(url)


In [ ]:
# Getting names of the countries

url = "https://en.wikipedia.org/wiki/List_of_international_airports_by_country"
response=requests.get(url)
print(response.status_code)


### Getting DFs with IATA and ICAO Codes

In [ ]:
# Getting codes
url = "http://www.flugzeuginfo.net/table_airportcodes_country-location_en.php"
iata_icao_dfs = pd.read_html(url)

len(iata_icao_dfs)
iata_icao_dfs[0]


### Getting Names and ICAO codes of Russian Airports

In [ ]:
# Getting tables with all Russian airports

ru_aiports_raw = pd.read_html("https://en.wikipedia.org/wiki/List_of_airports_in_Russia")

# We need 1st table because it contains the relevant data
ru_aiports_raw = ru_aiports_raw[0]

#df_printer(ru_aiports)

# Cleaning Data

## Create Dataframes
Combining tables with country names

Ps. Could have actually avoided using it because ```iata_icao_dfs``` have Country names corresponding with ICAO and IATA codes



###Find Country for each table

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')


# Extract all Countries before Oceania
countries = []

for level in soup.find_all('li',{'class':'toclevel-3'}):
  #hit = level.text.strip()
  country = level.find('span',{'class':'toctext'}).text.strip()
  countries.append(country)

print(len(countries))


# Extract Oceania countries
t1 = soup.find('li',{'class':"toclevel-1 tocsection-234"})

for level in t1.find_all('li',{'class':'toclevel-2'}):
  country = level.find('span',{'class':'toctext'}).text.strip()
  countries.append(country)

print(len(countries))

###Match airport codes with country naames

In [ ]:
# Remove table that is not the airport table
print(len(airport_dfs))
if len(airport_dfs) > 234:
  airport_dfs.pop(0)
print(len(airport_dfs))

# Cleaning Columns and Assigning country to the airport
i = 0 
modified_airport_dfs = []
for df in airport_dfs:
  if "IATA Code" in df.columns:
    df.rename(columns={"IATA Code": "IATA"},inplace=True)
  m_df = df[["IATA","Airport"]]

  # Assigning counttry
  m_df.insert(2, 'Country', countries[i])
  modified_airport_dfs.append(m_df)

  i+=1

# Creating 1 dataframe with all international airports, but without their codes
iata_df = pd.concat(modified_airport_dfs)
iata_df.reset_index(drop=True,inplace=True)
len(iata_df)
iata_df

In [ ]:
# Create table with all international airports and their codes

iata_icao_df = pd.concat(iata_icao_dfs)
iata_icao_df

###Internation Airports DF

In [ ]:

# Merging iata_df and iata_icao_df
int_airport_df = pd.merge(iata_df, iata_icao_df[['IATA','ICAO']], on='IATA', how="left") #check left and right merging

# Cleaning int_airport_df
INT_AIRPORTS = int_airport_df[['IATA','ICAO','Country','Airport']].drop_duplicates(subset=['IATA'], keep='first')
INT_AIRPORTS.reset_index(drop=True,inplace=True)

# Check for duplicates 
#INT_AIRPORTS['ICAO'].value_counts()
#df = INT_AIRPORTS.loc[INT_AIRPORTS['ICAO'] == 'UGGG']
#df


###DF with Russian Airports


In [ ]:
# Relevant Columns + Removing Nan in ICAO column
RU_AIRPORTS = ru_aiports_raw[["Airport name","ICAO"]].dropna(subset=['ICAO'])
RU_AIRPORTS["Country"] = "Russia"
RU_AIRPORTS.rename(columns={"Airport name": "Airport"},inplace=True)

#RU_AIRPORTS = RU_AIRPORTS.dropna(subset=['ICAO'])

# Dropping the rows with unused airports, with region names, and Simferopol, since it is not internation 
print("Size of df before:", len(RU_AIRPORTS))

discard = ["abandoned","closed","edit","Simferopol"]
RU_AIRPORTS = RU_AIRPORTS[~RU_AIRPORTS["Airport"].str.contains('|'.join(discard))]
RU_AIRPORTS.reset_index(drop=True,inplace=True)

# Check for duplicates
#print(RU_AIRPORTS['ICAO'].value_counts())

# Degugging
#df = RU_AIRPORTS.loc[RU_AIRPORTS['ICAO'] == 'UIII']
#print(df)

#Ozernaya and Sobolevo airports have the same ICAO code

print("Size of df before:", len(RU_AIRPORTS))

print(RU_AIRPORTS)
# To see names of the airports
#for row in RU_AIRPORTS["Airport"]:
# print(row)


###Flights Df

In [ ]:
# flights_jan
flights_jan = flights_jan[["origin","destination","day"]]
print(len(flights_feb))

# flights22
flights_feb = flights_feb[["origin","destination","day"]]
print(len(flights_feb))

# All flights df
flights = pd.concat([flights_jan,flights_feb])
print(len(flights))
print(flights.columns)


###Shorten flights DF with "origin" only containing Russian Airports 

In [ ]:
# Keep only flights with NaN and Russian Airport codes in "origin" column 
FLIGHTS = flights[flights['origin'].isin(RU_AIRPORTS['ICAO'])].copy()
print(len(flights))

# Drop NaN 
FLIGHTS.dropna(inplace=True)
print(len(FLIGHTS))

# Change 'day' from str to datetime
FLIGHTS['day'] = pd.to_datetime(FLIGHTS['day'], format="%Y/%m/%d").dt.date

#flights.head(50)

###Downloading Cleaned DFs


In [ ]:
#from google.colab import files

#FLIGHTS.to_csv("all_flights.csv")
#RU_AIRPORTS.to_csv("russian_airports.csv")
#INT_AIRPORTS.to_csv("international_airports.csv")


#files.download("all_flights.csv")
#files.download("russian_airports.csv")
#files.download("international_airports.csv")




# Creating Interface for people to work

*  Will be transferred to the Telegram Bot Functionality



In [ ]:
user_input = input("What Day and Airport in Russia? \n" +
                  "It accepts only 'dd mm yyyy' format. \nFor example: \n" +
                   "\t01-01-2022, Sheremetyevo\n" +
                   "\t01/01/2022, Sheremetyevo\n" 
                   "\t01-January-2022, Sheremetyevo\n" + 
                   "\t01 January 2022, Sheremetyevo:  " )



In [ ]:
RU_AIRPORTS_2=RU_AIRPORTS.copy()

def pop_printer(df,option:str):
  """Prints most travelled destinations or countries"""
  i=1
  for index,row in df.iterrows():
    if pd.isnull(row['Country']):
      continue

    if i == 1: suffix = "st"
    elif i == 2: suffix = "nd"
    elif i == 3: suffix = "rd"
    elif i == 4 or i == 5: suffix = "th"

    if option == "dest":
      print(f"**{i}{suffix} most traveled destination is {row['Airport']}, {row['Country']} with {row['Amount']} flight(s).")
    elif option == "country":
      print(f"**{i}{suffix} most traveled is country {row['Country']} with {row['Amount']} flight(s).")
      
    i+=1
    if i == 6: break
  
def pop_df_cleaner(df,option):
  """Creates lists with the most travelled destinations or countries to be plotted"""
  x,y = [],[]
  i=1 
  for index,row in df.iterrows():

    if pd.isnull(row['Country']):
      continue

    if option == "dest":
      if pd.isnull(row['Airport']):
        continue

    y.append(row['Amount'])
    if option == "country":
      x.append(row['Country'])
    elif option == "dest":
      x.append(row["Airport"]+', '+ row['Country'])

    i+=1
    if i == 6: break

  return x,y


def create_plt(df,option):
  """Creates the barplot with the most travelled destinations or countries"""
  x, y = pop_df_cleaner(df, option)
  fig = plt.figure()

  if option == "dest":
      plt.title('Destination Airports Vs Amount of Flights', fontsize=14, pad=20)
      plt.xlabel('Destination Airports', fontsize=11)

  elif option == "country":
      plt.title('Country Vs Amount of Flights', fontsize=14, pad=20)
      plt.xlabel('Country', fontsize=11)

  plt.ylabel('Amount of Flights', fontsize=10)
  plt.xticks(range(0,len(x)),x, fontsize=8, rotation=45)
  plt.margins(y=0.2)
  plt.bar(x=x, height=y)
  plt.show()




user_date_status, user_airport_status = False, False
while not user_date_status and not user_airport_status:
  user_date_status, user_airport_status = False, False
  # User input
  #user_input = "01 01 2022, Sheremetyevo" # "31 December 1900 Sheremetyevo" # "Abra"

  # Separate date and airport
  elements = user_input.split(',')

  # Not comma separated
  if len(elements) < 2:
    print('You forgot to add comma after date. Enter Date and Airport Again')
    continue

  # Missing airport or date
  if (elements[0] == "") or (elements[1] == ""):
    print('You forgot to add date or airport. Enter Date and Airport Again')
    continue

  # Cleaning date 
  user_date = elements[0]
  user_date.replace('-',' ').replace('/',' ')

  # Getting Date from User input
  pattern = r"\d{2} \d{2} \d{4}" + "|\d{2} \w+ \d{4}"
  match = re.search(pattern, user_date)

  # If invalid Date
  if not match:
    print('You entered invalid date. Enter Date and Airport Again')
    continue

  # Extracting all elements in the string
  month = user_date.split(' ')[1]

  # Converting to data
  if month.isdigit():
    date = datetime.strptime(match.group(), '%d %m %Y').date()
  else: 
    date = datetime.strptime(match.group(), '%d %B %Y').date()

  #print(date)
  if date:
    user_date_status = True

  # Airport
  user_airport = elements[1].strip()

  # Find ICAO code for airport
  ru_port = RU_AIRPORTS[RU_AIRPORTS["Airport"].str.contains(user_airport)]


  if ru_port.empty:
    print('You entered incorrect airport name. Enter Date and Airport Again')
    continue

  icao = ru_port.iloc[0][1]

  if icao:
    # can be a method

    # cleaning on the day of flight 
    
    ru_port_flights = FLIGHTS.loc[FLIGHTS['day'] == date]
    #print(FLIGHTS)
    #print(date)
    #print(FLIGHTS.loc[FLIGHTS['day'] == date])
    #print(ru_port_flights)

    # cleaning on origin Russian Airport and removing from-into flights
    ru_port_flights = ru_port_flights.loc[ru_port_flights['origin'] == icao]
    ru_port_flights = ru_port_flights.loc[ru_port_flights['destination'] != icao]


    ################## Statistics ##################

    #1
    num_flights = len(ru_port_flights) # Number of flights from airport
    if num_flights > 0:
      user_airport_status = True

    # df with unique destinations (not to use because does not have countiees)
    unique_dests_df = ru_port_flights["destination"].value_counts().rename_axis('ICAO').reset_index(name='Amount')
  
    #2
    num_unique_dests = unique_dests_df.shape[0] #alternatively len(unique_dests_df)

    #3 
    # Unique destinations
    most_visited_dests = INT_AIRPORTS.merge(unique_dests_df, on='ICAO', how='right')
    most_visited_dests = most_visited_dests.set_index('ICAO')
    RU_AIRPORTS_2 = RU_AIRPORTS_2.set_index('ICAO')
    most_visited_dests.update(RU_AIRPORTS_2)
    most_visited_dests.reset_index(inplace=True)
    RU_AIRPORTS_2.reset_index(inplace=True)
    most_visited_dests = most_visited_dests[["ICAO","Country","Airport","Amount"]]

 
    # Unique countries (omitting NaNs)
    most_visited_countries = most_visited_dests.loc[most_visited_dests["Country"] != "Russia"].dropna().copy()
    #print(most_visited_countries['Country'].unique())
    countries, amounts = [],[]
    for country in most_visited_countries['Country'].unique():
        countries.append(country)
        amount = 0
        for index, row in most_visited_countries.iterrows():
            if country == row['Country']:
                amount += row['Amount']
        amounts.append(amount)

    most_visited_countries = pd.DataFrame({'Country':countries,'Amount':amounts})

    #4
    num_unique_countries = most_visited_countries.shape[0] #alternatively len(unique_dests_df)




    ################## Output ##################
    #1
    print(f"*{num_flights} flight(s) were made from {ru_port.iloc[0][0]} on {date}.\n")
    if num_flights > 0:
      #2
      print(f"*To {num_unique_dests} unique destination(s).")

      #3
      pop_printer(df=most_visited_dests,option="dest")

      #3.1
      if num_unique_dests > 2:
        create_plt(most_visited_dests,option="dest")

      #4
      print(f"\n*To {num_unique_countries} unique countries.")

      #5
      pop_printer(df=most_visited_countries,option="country")

      #5.1
      if num_unique_countries > 2:
        create_plt(most_visited_countries,option="country")


# Desired Statistics:
# 1. Amount of flights from the airport
# 2. Amount  of unique destinations
# 3. To what destinations and airports and the count of flights to these destination
# 3-1. Diagram with the top travelled destinations (Matplotlib)
# 4. Amount of unique countries
# 5. To what countries and airports and the count of flights to these countries
# 5-1. Diagram with the top travelled countries (Matplotlib)

